# Model Context Protocol (MCP) Concepts

The **Model Context Protocol (MCP)** is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Think of it as a universal adapter that allows AI assistants to connect to various data sources and tools.

> 📝 **Hands-on Exercises**: After reviewing these concepts, complete the exercises in [EXERCISES.md](EXERCISES.md).

## Key Benefits
- **Standardization**: One protocol to connect to many tools
- **Security**: Controlled access to resources
- **Flexibility**: Works with any LLM provider

## MCP Architecture

```
+-----------------+     +-----------------+     +-----------------+
|   AI Agent      |---->|   MCP Client    |---->|   MCP Server    |
|   (Host)        |     |                 |     |   (Tools)       |
+-----------------+     +-----------------+     +-----------------+
```

### Components
1. **Host**: The AI application (e.g., Claude, ChatGPT integration, or custom agent)
2. **Client**: Connects to MCP servers on behalf of the host
3. **Server**: Exposes tools, resources, and prompts

## Transport Types

MCP supports two primary transport mechanisms:

### 1. STDIO (Standard Input/Output)
- Used for **local** MCP servers
- Server runs as a subprocess
- Communication via stdin/stdout
- Best for: Local tools, CLI applications

### 2. HTTP/SSE (Server-Sent Events)
- Used for **remote** MCP servers
- Server runs as a web service
- Communication via HTTP requests and SSE
- Best for: Cloud services, shared tools

## Workshop Projects Overview

| Project | Type | Transport | Description |
|---------|------|-----------|-------------|
| mcp_local_server | Local | STDIO | Python MCP server with Config and Ticket tools |
| mcp_remote_server | REST API | HTTP | Backend REST API for tickets |
| mcp_bridge | Remote | HTTP/SSE | MCP server that calls REST API |
| mcp_agent_client | Client | Both | AI agent that consumes MCP servers |

## Setup: Install Required Packages

Run the following cell to install the required packages for MCP and Azure OpenAI integration.

In [1]:
# Setup: Install required packages
# Run this cell first to ensure all dependencies are installed

import subprocess
import sys

packages = [
    "mcp",
    "httpx",
    "httpx-sse", 
    "openai",
    "azure-identity",
    "python-dotenv"
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [2]:
# Configuration: Load environment variables
# Set up your Azure OpenAI connection

import os
import json
from pathlib import Path
from dotenv import load_dotenv


def find_config_path(start_path: str) -> str:
    """Find the 'python' folder by traversing up from start_path."""
    current_dir = Path(start_path)
    
    while current_dir is not None:
        if current_dir.name.lower() == "python":
            return str(current_dir)
        if current_dir.parent == current_dir:
            break
        current_dir = current_dir.parent
    
    # Fallback to start path if python folder not found
    return start_path


def load_env_file(env_path: str) -> dict:
    """Load environment variables from .env file (JSON format)."""
    env_file = Path(env_path) / ".env"
    
    if not env_file.exists():
        return {}
    
    try:
        with open(env_file, 'r') as f:
            content = f.read()
            env_vars = json.loads(content)
            
            # Set environment variables
            for key, value in env_vars.items():
                os.environ[key] = str(value)
            
            return env_vars
    except json.JSONDecodeError:
        # Fallback: try loading as standard dotenv format
        load_dotenv(env_file, override=True)
        return {}
    except IOError as e:
        print(f"Warning: Failed to load .env file: {e}")
        return {}


# Load environment variables from .env file in the python root folder
config_path = find_config_path(os.getcwd())
env_vars = load_env_file(config_path)
if env_vars:
    print(f"✅ Loaded {len(env_vars)} environment variables from: {config_path}/.env")
else:
    # Fallback: try loading from current directory
    load_dotenv()
    print("⚠️ Loaded .env from current directory (fallback)")

# Check required configuration - support both naming conventions
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

if endpoint:
    print(f"✅ Azure OpenAI Endpoint: {endpoint}")
    print(f"✅ Deployment Name: {deployment}")
else:
    print("⚠️ Azure endpoint not set")
    print("Please set one of the following environment variables:")
    print("  - AZURE_AI_PROJECT_ENDPOINT or AZURE_OPENAI_ENDPOINT")
    print("And optionally:")
    print("  - AZURE_AI_MODEL_DEPLOYMENT_NAME or AZURE_OPENAI_DEPLOYMENT_NAME (default: gpt-4o-mini)")
    print("  - AZURE_OPENAI_API_KEY (for API key auth)")

# Check authentication method
api_key = os.getenv("AZURE_OPENAI_API_KEY")
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")

if api_key:
    print("✅ Authentication: API Key")
elif tenant_id and client_id and client_secret:
    print("✅ Authentication: Service Principal")
else:
    print("✅ Authentication: Azure CLI / DefaultAzureCredential")

⚠️ Loaded .env from current directory (fallback)
✅ Azure OpenAI Endpoint: https://proj-afp2-resource.openai.azure.com/
✅ Deployment Name: gpt-4.1-mini
✅ Authentication: API Key


In [3]:
# Create Azure OpenAI Client
# This demonstrates how to set up the client with different auth methods

import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, AzureCliCredential, ClientSecretCredential, get_bearer_token_provider

# Get configuration - AZURE_OPENAI takes priority over AZURE_AI_PROJECT
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini")

# Only fall back to Foundry endpoint if Azure OpenAI endpoint is not set
if not endpoint:
    endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", deployment)
    print("⚠️ Using Azure AI Foundry endpoint (AZURE_AI_PROJECT_ENDPOINT)")

if not endpoint:
    raise ValueError("Azure endpoint not set. Set AZURE_OPENAI_ENDPOINT environment variable.")

api_version = "2024-02-15-preview"

# Use API Key if provided (standard Azure OpenAI)
if api_key:
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version=api_version
    )
    print(f"✅ Created AzureOpenAI client with API Key auth")
    print(f"   Endpoint: {endpoint}")
    print(f"   Deployment: {deployment}")
else:
    # Token-based auth (for Foundry endpoints or when no API key)
    print("⚠️ No API key found, using token-based authentication")
    try:
        credential = AzureCliCredential()
        token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=token_provider,
            api_version=api_version
        )
        print("✅ Created AzureOpenAI client with Azure CLI auth")
    except Exception as e:
        credential = DefaultAzureCredential()
        token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=token_provider,
            api_version=api_version
        )
        print("✅ Created AzureOpenAI client with DefaultAzureCredential")

✅ Created AzureOpenAI client with API Key auth
   Endpoint: https://proj-afp2-resource.openai.azure.com/
   Deployment: gpt-4o-mini


In [4]:
# Debug: Check your configuration values
import os

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

print("=== Configuration Check ===")
print(f"Endpoint: {endpoint}")
print(f"API Key: {'*' * 10 + api_key[-4:] if api_key and len(api_key) > 4 else 'NOT SET or too short'}")
print(f"Deployment: {deployment}")

# Validate endpoint format
if endpoint:
    if not endpoint.startswith("https://"):
        print("⚠️ Endpoint should start with 'https://'")
    if not "openai.azure.com" in endpoint:
        print("⚠️ Endpoint should contain 'openai.azure.com'")
    if endpoint.endswith("/"):
        print("⚠️ Endpoint should NOT end with '/' - this can cause auth errors")
else:
    print("❌ Endpoint is not set!")

=== Configuration Check ===
Endpoint: https://proj-afp2-resource.openai.azure.com/
API Key: **********PLTk
Deployment: gpt-4.1-mini
⚠️ Endpoint should NOT end with '/' - this can cause auth errors


In [5]:
# Test the Azure OpenAI connection
# Send a simple completion request to verify everything is working

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say 'Hello from MCP Lab!' in exactly 5 words."}
    ],
    max_tokens=50
)

print("Response from Azure OpenAI:")
print(f"  {response.choices[0].message.content}")
print()
print(f"✅ Azure OpenAI connection successful!")
print(f"  Model: {response.model}")
print(f"  Tokens: {response.usage.total_tokens}")

Response from Azure OpenAI:
  Hello from the MCP Lab!

✅ Azure OpenAI connection successful!
  Model: gpt-4.1-mini-2025-04-14
  Tokens: 37


In [6]:
# MCP Server Example - Define tools
# This shows how to create MCP tools

from mcp.server import Server
from mcp.types import Tool, TextContent

# Create an MCP server instance
server = Server("example-server")

# Define a simple tool
@server.list_tools()
async def list_tools() -> list[Tool]:
    """List available tools."""
    return [
        Tool(
            name="GetConfig",
            description="Gets a configuration value by key",
            inputSchema={
                "type": "object",
                "properties": {
                    "key": {
                        "type": "string",
                        "description": "The configuration key"
                    }
                },
                "required": ["key"]
            }
        )
    ]

print("✅ MCP Server defined with tools")
print("Available tools:")
tools = await list_tools()
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

✅ MCP Server defined with tools
Available tools:
  - GetConfig: Gets a configuration value by key


## Closing Thoughts: From Concept to Implementation

Having explored the MCP concepts and the complete solution implementation, here are the key takeaways from this lab:

### Architecture Patterns You've Learned

The solution demonstrates a **three-tier MCP architecture**:

1. **Backend Layer** (`mcp_remote_server`) - A pure REST API using FastAPI that manages ticket data
2. **Bridge Layer** (`mcp_bridge`) - An HTTP/SSE MCP server that wraps the REST API, exposing it through MCP protocol
3. **Local Tools Layer** (`mcp_local_server`) - A STDIO MCP server providing direct configuration and ticket management

### Transport Mechanisms in Action

You've worked with both MCP transport types:

- **STDIO Transport** (Local Server): Spawns Python subprocess, communicates via stdin/stdout. Perfect for local tools and CLI integrations.
- **HTTP/SSE Transport** (Bridge): Runs as web service, enables remote access. Ideal for cloud services and shared tooling across multiple agents.

### AI Agent Integration Pattern

The `mcp_agent_client` demonstrates production-ready patterns:

- **Multi-auth Support**: API Key, Service Principal (ClientSecretCredential), and Azure CLI fallback
- **Interactive Sessions**: Chat loop with tool calling, maintaining conversation history
- **Tool Discovery**: Dynamic tool listing and OpenAI function format conversion
- **Error Handling**: Graceful degradation when services aren't available
- **Menu-driven UX**: Clean separation between local and remote MCP demos

### Real-World Application

This architecture pattern enables:

- **Existing API Integration**: Wrap any REST API with MCP without modifying the backend
- **Hybrid Deployments**: Combine local tools (config management) with remote services (ticket systems)
- **Agent Portability**: Same MCP client works with both STDIO and HTTP/SSE transports
- **Tool Composition**: Agents can discover and use tools dynamically at runtime

### Next Steps in Your Journey

1. **Extend the Local Server**: Add more tools (file operations, system commands, database queries)
2. **Secure the Bridge**: Add authentication, rate limiting, and request validation
3. **Multi-Server Agents**: Connect your agent to multiple MCP servers simultaneously
4. **Custom Protocols**: Bridge other protocols (GraphQL, gRPC, WebSockets) to MCP

### Key Takeaway

**MCP is a universal adapter layer** that lets AI agents interact with any tool or data source through a standardized protocol. By separating concerns (backend logic, MCP exposure, agent interaction), you create maintainable, scalable, and composable AI systems.

The code examples in this notebook provide the foundation. The solution in `../solution/` shows how these pieces come together into a complete, production-ready system with proper error handling, authentication, and user experience.